# Diffusion coeefficient analysis 
This notebook performs a diffusion coefficient analysis and adds the resulting D profiles to the shell files that are input to NuGrid's `mppnp` code for optional post-processing with diffusive mixing (instead of ATS mixing). 

# Determine D for all dumps and write D_vr and D_composite into shell files

A variation of Lagrangian_diffusion_analysis creating shell files for all dumps.

In [1]:
%pylab ipympl
# %pylab inline
import sys, os

from nugridpy import ascii_table as asct
from ppmpy import ppm
from matplotlib import pyplot as plt
from nugridpy import utils
from nugridpy import astronomy as ast
from nugridpy import nugridse as nuse
import numpy as np
import logging
import pickle
from scipy import interpolate

mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)
cb = utils.colourblind

# figure sizes
stdSize = 4.
stdRatio = 1.61803398875

Populating the interactive namespace from numpy and matplotlib


DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


### Hydro data

In [2]:
asdr_dir = '/data/ASDR/PPMstar/'
asdr = '/data/ASDR/PPMstar/LowZRAWD/'
rprof_paths = {'N15':asdr+'N15-LowZRAWD-768-10x-burn-moms/prfs/',\
               'N16':asdr+'N16-LowZRAWD-1536-10x-burn-moms/prfs/',\
               'N17':asdr+'N17-LowZRAWD-1152-100x-burn-moms/prfs/'}

In [3]:
rprof_paths

{'N15': '/data/ASDR/PPMstar/LowZRAWD/N15-LowZRAWD-768-10x-burn-moms/prfs/',
 'N16': '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/prfs/',
 'N17': '/data/ASDR/PPMstar/LowZRAWD/N17-LowZRAWD-1152-100x-burn-moms/prfs/'}

In [4]:
runs = ['N15','N16','N17']
rp_sets = {rid:ppm.RprofSet(path,verbose=0) for rid, path in rprof_paths.items()}

In [5]:
# Set directory base name of astrophysical simulation data repository, 
# the required data is held in two places
asdr_dir = '/data/ASDR/PPMstar/'
# asdr_dir = '/data/niagara_project/projects/'


# N16 

In [6]:
runid='N16'
shell_files_dir = asdr_dir+'3D1D-advection/N16_shell_files/N16-standard-mppnp/'

In [7]:
# read convective boundary from shell file
dump = 100
shell_file = 'N16-standard-mppnp-'+str(dump).zfill(4)+'.shell'
shell_file = open(shell_files_dir+shell_file)
[shell_file.readline() for x in range(7)]
r_cb_shell_bot = float(shell_file.readline().split()[0])
r_cb_shell_top = float(shell_file.readlines()[-1].split()[0])
close()
r_cb_shell_top_0 = r_cb_shell_top  # for dump = 0 -- needed by DsolveLagr

In [8]:
# dump = 1035
dump = 650
shell_file = 'N16-standard-mppnp-'+str(dump).zfill(4)+'.shell'
shell_file = open(shell_files_dir+shell_file)
[shell_file.readline() for x in range(7)]
r_cb_shell_bot = float(shell_file.readline().split()[0])
r_cb_shell_top = float(shell_file.readlines()[-1].split()[0])
close()

### D(vr)

In [9]:
# get data for D_Jones plot
Hp = rp_sets[runid].compute('Hp',dump)
R  = rp_sets[runid].get('R',dump)
Ur = rp_sets[runid].compute('|Ur|',dump)

# conv boundaries from rprof
# rtop = rp_sets[runid].bound_rad(dump,20.,23)
# rbot = rp_sets[runid].bound_rad(dump,6.,9.,criterion='max_grad')

# conv boundaries from shell file
rtop = r_cb_shell_top
rbot = r_cb_shell_bot

# find convection zone
inds_conv = where( (R < rtop) * (R > rbot))[0]
mlen = Hp[inds_conv]
inds=where(rtop-R[inds_conv] < Hp[inds_conv])[0]
mlen[inds] = rtop-R[inds_conv][inds]
inds = where(R[inds_conv]-rbot < Hp[inds_conv])[0]
mlen[inds] = R[inds_conv][inds]-rbot

# D_Jones
alpha_MLT = 1.6
Dvr = (1./3)* alpha_MLT * mlen * Ur[inds_conv]  *1.e16  # Mm**2/s

### Test plot

In [10]:
for i in range(1000):
    close(i)
close(125);fig=figure(125)
i=2
ax1 = fig.add_subplot(111)

ax1.plot(R[inds_conv],log10(Dvr),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
     label='$D_\mathrm{vr}$'); i+=1
ax1.set_ylabel('y1')
ax1.set_ylabel('$\log D / \mathrm{[cm^2/s]}$')
ax1.set_ylim(10,None); ax1.set_xlim(7,25)
ax1.set_xlabel('$R/\mathrm{Mm}$')

ax2 = ax1.twinx()
ax2.plot(R[inds_conv],Hp[inds_conv],utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
       label='$H_p$'); i+=1
ax2.set_ylabel('$H_\mathrm{p}$')
plt.title(runid+' dump '+str(dump))

plt.legend(frameon=False,loc=8)
plt.show()
savefig(runid+'1dump'+str(dump)+".pdf")
print('D_Lagr for dump:'+str(dump))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

D_Lagr for dump:650


### Time for diffusion to cross mixing length
I need to take the difference of two averages of FV. For that I determine the diffusion time to diffuse one mixing length. I go back in dump number by that time and take the FV average over one diffusion time. I do the same in the future direction. I am determining D then from the difference of these two averaged FV profiles, each which is averaged over a diffusion time  and appart by one diffusion time. 
$$
\alpha H_\mathrm{p} = 2 \sqrt{D \Delta t} \\
\Delta t = \frac{(\alpha H_\mathrm{p})^2}{4D}
$$
Take $H_\mathrm{p}$ and $D$ where $D$ is maximum.

In [11]:
ind_max_Dvr = where(max(Dvr)==Dvr)[0][0]
Hp_max_Dvr = Hp[inds_conv][ind_max_Dvr]
Dmax = max(Dvr) * 1e-16 # code units!
tsec_Dmax = (alpha_MLT*Hp_max_Dvr)**2/(4*Dmax)  # diffusion time scale in seconds

In [12]:
print(Dmax,tsec_Dmax/60.)

0.028613093431142243 2.565771760728137


In [13]:
rp_hst = rp_sets[runid].get_history()
time_secs = rp_hst.get('time(secs)')

# dt at dump 
thisrp = rp_sets[runid].get_dump(dump)
dt = thisrp.get('dtinit')*thisrp.get('NcyclesPerDump')

# tsec_Dmax is equivalent to ndndump many dumps
ndndump = int(np.round(tsec_Dmax/dt))

#dump for tsec_dump - tsec_Dmax
dump_lower = dump - ndndump
dump_upper = dump + ndndump

print(dump_lower,dump,dump_upper)

644 650 656


### D(FV)

In [14]:
res = {}
Ds = {}

In [15]:
#%--no-display
# Burn on
ifig0 = 11
# d0 = rp_sets[runid].get_dump_list()[-1] - 400

runid_case = runid+str(dump)
d0 = dump
d1 = dump_lower
d2 = dump_upper
delta_d1 = d0 - d1
delta_d2 = d2 - d0
dr_ov_estimate = 0.1*Hp[where(r_cb_shell_top_0 < R)[0][-1]]
rlim_max = r_cb_shell_top_0 + dr_ov_estimate
T9corr_params = {'kind':1, 'params':{'a':0.46, 'b':0.77}}
#T9corr_params = {}
print(d1 - delta_d1, d0 + 1,d0, d2 + delta_d2 + 1)
res[runid] = rp_sets[runid].DsolveLgr(\
              range(d1 - delta_d1, d0 + 1), \
              range(d0, d2 + delta_d2 + 1), \
              var='Xcld', \
              data_rlim = (11.1, rlim_max), \
              show_plots=True, \
              ifig0=ifig0, \
              run_id=runid, \
              logmt=True, \
              mtlim=(1e-2, 1e-5), \
              rlim=(10, rlim_max+2), \
              plot_var=True, \
              logvar=True, \
              varlim=(1e-9, 2e0), \
              sigmalim=(1e50, 1e62), \
              Dlim=(1e9, 1e17), \
              fit_rlim=None, \
              integrate_upwards=True, \
              src_func=rp_sets[runid].compute_Xdot_C12pg, \
              src_args={'T9corr_params':T9corr_params})

638 651 650 663


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
r = 0.5*(res[runid]['r1']+res[runid]['r2'])
D = res[runid]['D']
Dplus = D[where(D>0)[0]]
rplus = r[where(D>0)[0]]

### Top panel Figure 1

In [17]:
close(1251);fig=figure(1251,figsize=(stdRatio*stdSize,stdSize))
i=2
ax1 = fig.add_subplot(111)

ax1.plot(R[inds_conv],(Dvr),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
     label='$D_\mathrm{vr}$'); i+=1
ax1.plot(rplus,(Dplus),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
     label='$D_\mathrm{FV}$'); i+=1
plt.legend(frameon=False,loc=8)
ax1.set_ylabel('y1')
ax1.set_yscale('log')
ax1.set_ylabel('D / cm$^2$ s$^{-1}$')
ax1.set_ylim(5*10**11,5*10**15); ax1.set_xlim(7,25)
ax1.set_xlabel('r / Mm')

ax2 = ax1.twinx()
ax2.plot(R[inds_conv],Hp[inds_conv],utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
       label='H$_P$'); i+=1
ax2.set_ylabel(r'H$_P$ / Mm')

plt.title("")
plt.legend(frameon=False,loc=1)
plt.tight_layout()
plt.show()
savefig(runid+'-DHp-compare-dump'+str(dump)+".pdf")
print('D_Lagr for dump:'+str(dump))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

D_Lagr for dump:650


In [18]:
# gr = 1.61803398875;size=4
# close(1251);fig=figure(1251,figsize=(gr*size,size))
# i=2
# ax1 = fig.add_subplot(111)

# ax1.plot(R[inds_conv],log10(Dvr),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
#      label='$D_\mathrm{vr}$'); i+=1
# ax1.plot(rplus,log10(Dplus),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
#      label='$D_\mathrm{FV}$'); i+=1
# plt.legend(frameon=False,loc=8)
# ax1.set_ylabel('y1')
# ax1.set_ylabel('$\log D / \mathrm{[cm^2/s]}$')
# ax1.set_ylim(11,None); ax1.set_xlim(7,25)
# ax1.set_xlabel('$R / \mathrm{[Mm]}$')

# ax2 = ax1.twinx()
# ax2.plot(R[inds_conv],Hp[inds_conv],utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
#        label='$H_p$'); i+=1
# ax2.set_ylabel('$H_\mathrm{p} / \mathrm{[Mm]}$')

# plt.legend(frameon=False,loc=1)
# plt.tight_layout()
# plt.show()
# savefig(runid+'-DHp-compare-dump'+str(dump)+".pdf")
# print('D_Lagr for dump:'+str(dump))


### Fuse D(vr) and D(FV)

We want to have the $D_\mathrm{FV}$ coefficient in the top half where the entrainment takes place. In the lower part of the convection zone the entrained fluid is burning and it can obviously not be used to determine the D coefficient. There we take the maxumum of $D_\mathrm{FV}$  and $D_\mathrm{vr}$. The separation between the two regimes is where the maximum of $D_\mathrm{FV}$  is located.

In [19]:
# f_Dvr  = interpolate.interp1d(R[inds_conv],Dvr, kind='linear',fill_value=ones(1))
# f_DFV  = interpolate.interp1d(rplus,Dplus, kind='linear',fill_value=ones(1))

# fill_value option seems not to work as advertised, so we do this:
f_Dvr  = interpolate.interp1d(R[inds_conv],Dvr, kind='linear')
f_DFV  = interpolate.interp1d(rplus,Dplus, kind='linear')
def interp(f_DFV,rr,fill_value=1.):
    '''Interpolate if in range, else assign fill_value'''
    d = []
    for thisr in rr: 
        try:
            d.append(f_DFV(thisr))
        except ValueError:
            d.append(fill_value)
    return array(d)

In [20]:
max(Dplus[(rplus > 13.0)])

929552211809495.4

In [21]:
# max location of Dplus
ind_max_Dplus = np.where(max(Dplus)==Dplus)[0][0]
inds_R_plusmax = np.where(R > rplus[ind_max_Dplus])

In [22]:
# combine as described above
Dcomb = maximum(interp(f_DFV,R),interp(f_Dvr,R))
Dcomb[inds_R_plusmax] = interp(f_DFV,R[inds_R_plusmax])

In [23]:
close(119);figure(119)
i=2
plot(R[inds_conv],log10(Dvr),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
     label='$D_\mathrm{vr}$'); i+=1

plot(rplus,log10(Dplus),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
      label='$D_\mathrm{FV}$'); i+=1

plot(R[inds_conv],log10(Dcomb[inds_conv]),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
      label='$D_\mathrm{combined}$' )

print('D_Lagr for dump:'+str(dump))
title(runid+' dump '+str(dump))
xlabel('$R/\mathrm{Mm}$');ylabel('$\log D / \mathrm{[cm^2/s]}$')
ylim(10,None),xlim(7,25),legend(frameon=False)
savefig(runid+'1dump'+str(dump)+".pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

D_Lagr for dump:650


## Read shell file and write with two D columns added

In [24]:
# read shell file: just read the headerlines 
shell_file = 'N16-standard-mppnp-'+str(dump).zfill(4)+'.shell'
shell_fobj = open(shell_files_dir+shell_file)
header_lines = [shell_fobj.readline().rstrip('\n') for x in range(6)]
shell_fobj.close()

In [25]:
# read columns of shell file
# `_s` indicates shell file column variable
r_s, rho_s, v_s, m_s, T9_s, gamma_s, fu_s, fd_s = loadtxt(shell_files_dir+shell_file,skiprows=7,unpack=True) 

The shell file grid has lower resolution compared to the ppmstar grid. The shell files are supposed to be half of the ppmstar grid (except for N16 where it is at the N17's resolution to make the full network calculations faster). This is because we use moments data for gamma which is 1/4 the ppmstar grid. This is the only quantity that is calibrated with the moments data.

In [26]:
print('N16: Shell file grid zones: {:3d}; convection zone PPMstar grid: {:3d}.'.format(len(r_s),len(R[inds_conv][::-1])))

N16: Shell file grid zones: 257; convection zone PPMstar grid: 348.


In [27]:
# and create an interpolation function for writing shell files which 
# are on a different grid
f_Dcomb  = interpolate.interp1d(R[inds_conv],log10(Dcomb[inds_conv]), kind='linear',fill_value='extrapolate')
Dcomb_s = [10**(f_Dcomb(r)) for r in r_s]
f_Dvr  = interpolate.interp1d(R[inds_conv],log10(Dvr), kind='linear',fill_value='extrapolate')
Dvr_s = [10**(f_Dvr(r)) for r in r_s]

In [28]:
dcols=('r', 'rho', 'v', 'm', 'T9', 'gamma', 'fu', 'fd', 'Dvr', 'Dvr+FV')
# take care of D arrays being in reverse order, shell files go from the inside out
data = [r_s, rho_s, v_s, m_s, T9_s, gamma_s, fu_s, fd_s, Dvr_s, Dcomb_s]
shell_file = 'N16-wD-'+str(dump).zfill(4)+'.shell'
# asct.write(shell_file, header_lines, dcols, data, sldir='.', sep='', trajectory=False, download=False,
#            data_fmt='{:18.12e} ',overwrite=True)
asct.write(shell_file, header_lines, dcols, data, sldir='.', sep='', trajectory=False, download=False,
           data_fmt='{:18.12e} ')

Warning this method will overwrite ./N16-wD-0650.shell
Would you like to continue? (y)es or (n)no?


-->  n


No Selected
Returning None


### Test: Read and plot

In [35]:
sf = asct.readTable(shell_file,sldir='.')

In [36]:
sf.hattrs

['runid = N16',
 'runDump = 650',
 'runTime(secs) = 1.793778320000e+04',
 'entrainmentRate = 1.683380753786e-05',
 'timestep = 2.758980000000e+01',
 'subtimesteps = 19']

In [37]:
sf.dcols

['r', 'rho', 'v', 'm', 'T9', 'gamma', 'fu', 'fd', 'Dvr', 'Dvr+FV']

In [38]:
ifig=197;close(ifig);figure(ifig)
sf.plot('r','Dvr',shape='--',legend='Dvr',logy=True)
sf.plot('r','Dvr+FV',shape='-',legend='Dvr+FV',logy=True,limits=[7.5,24.3,10,15.5])
savefig(shell_file+'.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
# sf.get('Dvr+FV')  
# sf.get will be returning something different if I plot with logy=True before --- FIX THIS!!!
# That is why the following plot will not work unless you rerun
sf = asct.readTable(shell_file,sldir='.')

In [40]:
close(1985)
figure(1985)
plot(sf.get('r'),log10(sf.get('Dvr+FV')),label='Dvr+FV')
plot(R[inds_conv],log10(Dcomb[inds_conv]),utils.linestylecb(i)[0],color=utils.colourblind(i-2),\
      label='$D_\mathrm{combined}$' )
legend(loc=0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …